In [1]:
import requests
from pathlib import Path

In [ ]:
API_SEARCH_URL = "https://docslook.interrao.ru/api/search"
API_UPLOAD_URL = "http://localhost:8000/api/upload"
API_KEY = "key1"   # ваш ключ из .env → API_SECRET или API_KEYS

In [3]:
# Пути к файлам, которые хотите загрузить
files_to_upload = list( Path('./doc').glob('test.*') )   

In [5]:
# Формируем кортежи (имя параметра, (имя файла, бинарные данные, mime-type))
files = [
    ("files", (f.name, open(f, "rb"), "application/octet-stream"))
    for f in files_to_upload
]

headers = {"X-API-Key": API_KEY}

response = requests.post(API_UPLOAD_URL, headers=headers, files=files)

if response.ok:
    data = response.json()
    print("Job ID:", f'http://localhost:8000/jobs/{data["job_id"]}')
    print("Queued:", data["queued"])
    print("Prefix:", data["prefix"])
else:
    print("Ошибка:", response.status_code, response.text)

Job ID: http://localhost:8000/jobs/5320ef2d166f44038d25731ee38cc644
Queued: 5
Prefix: 2025101220_api


In [ ]:
import requests
import time

# 1. Загрузка
headers = {"X-API-Key": "supersecret1"}
with open("document.pdf", "rb") as f:
    response = requests.post(
        "http://localhost:8000/api/upload",
        headers=headers,
        files={"files": f},
        data={"owner": "test", "owner_email": "test@example.com"}
    )

job_id = response.json()["job_id"]
print(f"Uploaded, job_id: {job_id}")

# 2. Ожидание завершения OCR
while True:
    status_resp = requests.get(f"http://localhost:8000/api/jobs/{job_id}", headers=headers)
    status = status_resp.json()
    
    print(f"Status: {status['status']}, Progress: {status['progress']}%")
    
    if status["status"] in ["completed", "error"]:
        break
    
    time.sleep(5)

# 3. Поиск по распознанному тексту
search_resp = requests.post(
    "http://localhost:8000/api/search",
    headers=headers,
    json={"q": "keyword", "limit": 10}
)
results = search_resp.json()
print(f"Found {results['total']} documents")